# Part 4: Using GPU acceleration with PyTorch

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 138 kB 5.2 MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [ ]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-0.5014, -0.4329, -0.1969,  ...,  0.0853,  0.4672,  0.0312],
        [ 0.7237,  0.1092,  2.7821,  ..., -0.8004, -0.6946,  0.7398],
        [ 0.0196,  1.0539,  0.5963,  ...,  0.0029,  0.0693, -0.4659],
        ...,
        [-0.2030, -1.5310,  1.0058,  ...,  1.6961, -0.3776, -0.1184],
        [-0.1058,  0.6591,  0.0806,  ...,  0.1103, -0.8577,  1.8060],
        [-0.6867, -1.7262, -0.0242,  ...,  0.0932,  0.0480,  0.1398]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [ ]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-0.5482,  2.4852,  2.3602,  ..., -0.1924, -1.1044,  0.7275],
        [ 1.5021, -0.4546, -0.0109,  ..., -0.1776,  0.9714, -0.3464],
        [ 0.8554,  0.0591,  0.0609,  ..., -0.0531,  0.8123,  0.0182],
        ...,
        [ 0.3212,  0.3898,  1.7511,  ..., -3.3504,  0.0483, -1.0660],
        [-0.7073,  1.0928,  0.0620,  ...,  0.1184,  0.1711, -0.1249],
        [-0.7726,  0.8821,  0.1511,  ..., -1.9859,  0.1895, -0.5283]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-1.1157e+00, -1.9985e-02, -1.7683e+00,  ..., -4.0983e-01,
          2.9888e-01,  4.8921e-01],
        [-1.4913e+00, -5.2062e-02,  4.0457e-01,  ..., -3.6704e-01,
          7.5040e-01, -6.7964e-01],
        [ 7.5003e-01, -1.1001e+00, -1.4353e+00,  ..., -4.7435e-01,
         -1.3017e+00, -1.5519e-01],
        ...,
        [-5.6080e-01, -3.1275e-01,  3.0676e-01,  ..., -5.3519e-01,
          1.1762e-03, -2.9370e-01],
        [-1.4625e-01, -5.4381e-01, -3.6473e-01,  ...,  1.6968e-01,
          2.9731e-01,  1.7560e-02],
        [-1.5467e+00,  7.2321e-02,  8.9900e-01,  ..., -1.1242e+00,
         -1.3891e+00, -2.1678e+00]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?


GPUs are bandwidth optimized. CPUs are latency(memory access time) optimized.

Memory Bandwidth:
Bandwidth is one of the main reasons why GPUs are faster for computing than CPUs.
Due to large datasets,the CPU takes up a lot of memory while training the model.
The standalone GPU, on the other hand, comes with a dedicated VRAM memory. Thus, CPU’s memory can be used for other tasks. But, transferring large chunks of memory from CPU to GPU is a bigger challenge.

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

One potential problem that can arise is that data allocated to the GPU could exceed device memory capacity. Workload sizes beyond hundreds of GB cannot be fully contained in even high-end GPU devices like the Tesla P100, which only possess a mere 16 GB of memory capacity.